In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

In [2]:
df = pd.read_csv("diabetes.csv")
pd.set_option("display.MAX_COLUMNS", None)

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()["Outcome"]

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [7]:
X_scaled = sc.fit_transform(X)

In [8]:


X_scaled.shape

(768, 8)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size = 0.2, random_state = 2)

In [10]:
X_train.shape

(614, 8)

In [11]:
model = Sequential()

In [12]:
model.add(Dense(32, activation ="relu", input_dim = 8))
model.add(Dense(16, activation = "relu"))
model.add(Dense(1, activation = "sigmoid"))

In [13]:
model.compile(optimizer = "Adam", metrics = ["accuracy"], loss = "binary_crossentropy")

In [14]:
history = model.fit( X_train, Y_train, batch_size = 32,epochs = 100, validation_data = (X_test, Y_test))

Epoch 1/100
20/20 [==============================] - 2s 15ms/step - loss: 0.6874 - accuracy: 0.6352 - val_loss: 0.6068 - val_accuracy: 0.6948
Epoch 2/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6155 - accuracy: 0.6531 - val_loss: 0.5656 - val_accuracy: 0.7143
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5686 - accuracy: 0.6906 - val_loss: 0.5371 - val_accuracy: 0.7403
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5344 - accuracy: 0.7215 - val_loss: 0.5125 - val_accuracy: 0.7662
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5093 - accuracy: 0.7329 - val_loss: 0.4969 - val_accuracy: 0.7468
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4918 - accuracy: 0.7443 - val_loss: 0.4900 - val_accuracy: 0.7468
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4789 - accuracy: 0.7573 - val_loss: 0.4842 - val_accuracy: 0.7532
Epoch 8/100


In [15]:
# How to select appropriate optimizer
# No. of nodes in a layer
# How to select no. of hidden layers
# All in one model

In [16]:
!pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 7.5 MB/s eta 0:00:00


In [17]:
import kerastuner as kt

Using TensorFlow backend


<ipython-input-17-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


# How to select appropriate optimizer

In [18]:
def build_model(hp): # hp is Hyper Parameter
  model = Sequential()
  model.add(Dense(32, activation ="relu", input_dim = 8))
  model.add(Dense(1, activation = "sigmoid"))

  optimizer = hp.Choice("optimizer", values = ["rmsprop", "adam","sgd","adadelta"])

  model.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = ["accuracy"])

  return model

In [19]:
# RandomSearch is like GridSearchCV in machine learning
tuner = kt.RandomSearch(build_model,
                        max_trials = 5,
                        objective = "val_accuracy")

In [20]:
# Search will search for accuracy of all paramters in optimizer
tuner.search(X_train, Y_train, epochs = 5, validation_data = (X_test, Y_test))

Trial 4 Complete [00h 00m 04s]
val_accuracy: 0.7597402334213257

Best val_accuracy So Far: 0.7597402334213257
Total elapsed time: 00h 00m 10s


In [21]:
# To get best optimizer
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [22]:
# To get best model where num_model =1 indicates number of best models and here it is 1 means the best one.
model = tuner.get_best_models(num_models=1)[0]

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [24]:

# initial epochs means it will start from 6 and we are doing so because our model has already ran for 5 epochs in keras tuner
model.fit(X_train, Y_train, epochs = 100, batch_size = 32, initial_epoch = 6, validation_data = (X_test, Y_test))

Epoch 7/100
20/20 [==============================] - 2s 19ms/step - loss: 0.5287 - accuracy: 0.7459 - val_loss: 0.4926 - val_accuracy: 0.7727
Epoch 8/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5153 - accuracy: 0.7557 - val_loss: 0.4840 - val_accuracy: 0.7532
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5061 - accuracy: 0.7590 - val_loss: 0.4772 - val_accuracy: 0.7662
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4974 - accuracy: 0.7638 - val_loss: 0.4722 - val_accuracy: 0.7727
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4904 - accuracy: 0.7655 - val_loss: 0.4674 - val_accuracy: 0.7792
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4843 - accuracy: 0.7687 - val_loss: 0.4636 - val_accuracy: 0.7727
Epoch 13/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4792 - accuracy: 0.7752 - val_loss: 0.4583 - val_accuracy: 0.7857
Epoch 14

# No. of nodes in a layer


In [25]:
# # To delete untitled folder created during training
# import shutil

# shutil.rmtree('untitled_project')

In [26]:
def build_model(hp):
  units = hp.Int(name = "units",min_value = 8, max_value = 128,step = 8) # we can remove this step and by default it will have value as 1
  model = Sequential()
  model.add(Dense(units=units, activation = "relu", input_dim = 8))
  model.add(Dense(1, activation = "sigmoid"))
  # optimizer = hp.Choice("optimizer", values = ["rmsprop", "adam", "adadelta", "sgd"])

  model.compile(optimizer = "rmsprop", loss = "binary_crossentropy", metrics = ["accuracy"])
  return model

In [27]:
tuner2 = kt.RandomSearch(build_model, max_trials = 5, objective="val_accuracy",directory = "some_directory", project_name = "Akshay kumar")

In [28]:
tuner2.search(X_train, Y_train, epochs = 5, validation_data = (X_test, Y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 18s


In [29]:
tuner2.get_best_hyperparameters()[0].values

{'units': 120}

In [30]:
model2 = tuner2.get_best_models(num_models = 1 )[0]

In [31]:
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               1080      
                                                                 
 dense_1 (Dense)             (None, 1)                 121       
                                                                 
Total params: 1,201
Trainable params: 1,201
Non-trainable params: 0
_________________________________________________________________


In [32]:
model2.fit(X_train, Y_train, epochs = 100,initial_epoch = 6,  validation_data = (X_test, Y_test))

Epoch 7/100
20/20 [==============================] - 1s 19ms/step - loss: 0.4617 - accuracy: 0.7866 - val_loss: 0.4601 - val_accuracy: 0.7597
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4565 - accuracy: 0.7866 - val_loss: 0.4581 - val_accuracy: 0.7662
Epoch 9/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4523 - accuracy: 0.7785 - val_loss: 0.4570 - val_accuracy: 0.7532
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4497 - accuracy: 0.7850 - val_loss: 0.4611 - val_accuracy: 0.7792
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4474 - accuracy: 0.7818 - val_loss: 0.4618 - val_accuracy: 0.7792
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4453 - accuracy: 0.7818 - val_loss: 0.4629 - val_accuracy: 0.7857
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4431 - accuracy: 0.7850 - val_loss: 0.4652 - val_accuracy: 0.7857
Epoch 14

# How to select no. of hidden layers

In [33]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(72, activation = "relu", input_dim = 8))
  for i in range(hp.Int(name = "num_layers", min_value = 1, max_value =10)):
    model.add(Dense(72, activation = "relu"))
  model.add(Dense(1, activation = "sigmoid"))
  model.compile(optimizer = "rmsprop", loss = "binary_crossentropy", metrics = ["accuracy"])
  return model

In [34]:
tuner3 = kt.RandomSearch(build_model, max_trials = 5, objective="val_accuracy",directory = "my_dir2",project_name = "srk")

In [35]:
tuner3.search(X_train, Y_train, epochs = 5, validation_data = (X_test, Y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 14s


In [36]:
tuner3.get_best_hyperparameters()[0].values

{'num_layers': 7}

In [37]:
model3 = tuner3.get_best_models(num_models = 1 )[0]

In [38]:
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 72)                648       
                                                                 
 dense_1 (Dense)             (None, 72)                5256      
                                                                 
 dense_2 (Dense)             (None, 72)                5256      
                                                                 
 dense_3 (Dense)             (None, 72)                5256      
                                                                 
 dense_4 (Dense)             (None, 72)                5256      
                                                                 
 dense_5 (Dense)             (None, 72)                5256      
                                                                 
 dense_6 (Dense)             (None, 72)                5

In [39]:
model3.fit(X_train, Y_train, epochs = 100, initial_epoch = 6,validation_data = (X_test, Y_test))

Epoch 7/100
20/20 [==============================] - 2s 26ms/step - loss: 0.4263 - accuracy: 0.8013 - val_loss: 0.6502 - val_accuracy: 0.6753
Epoch 8/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4153 - accuracy: 0.8111 - val_loss: 0.5163 - val_accuracy: 0.7273
Epoch 9/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4075 - accuracy: 0.7980 - val_loss: 0.4800 - val_accuracy: 0.7857
Epoch 10/100
20/20 [==============================] - 0s 7ms/step - loss: 0.3966 - accuracy: 0.8029 - val_loss: 0.5008 - val_accuracy: 0.7662
Epoch 11/100
20/20 [==============================] - 0s 8ms/step - loss: 0.3854 - accuracy: 0.8241 - val_loss: 0.4779 - val_accuracy: 0.7532
Epoch 12/100
20/20 [==============================] - 0s 8ms/step - loss: 0.3639 - accuracy: 0.8306 - val_loss: 0.5408 - val_accuracy: 0.7597
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3504 - accuracy: 0.8388 - val_loss: 0.5330 - val_accuracy: 0.7597
Epoch 14

# All in one model

In [50]:
def build_model(hp):
  model = Sequential()
  counter = 0
  for i in  range(hp.Int(name = "min_layers", min_value = 1, max_value = 10)):
    if counter == 0:
      model.add(Dense(units = hp.Int("units"+str(i),min_value = 8, max_value = 128, step = 8), activation = hp.Choice("activation"+str(i), values = ["relu", "tanh", "sigmoid"]), input_dim = 8))
      model.add(Dropout(hp.Choice("Dropout"+str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(Dense(units = hp.Int("units"+str(i),min_value = 8, max_value = 128, step = 8), activation = hp.Choice("activation"+str(i), values = ["relu", "tanh", "sigmoid"])))
      model.add(Dropout(hp.Choice("Dropout"+str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter += 1
  model.add(Dense(units = 1, activation = "sigmoid"))
  model.compile(optimizer = hp.Choice("otimizer", values = ["rmsprop", "adam", "adadelta", "sgd"]), loss = "binary_crossentropy", metrics= ["accuracy"])
  return model

In [51]:
tunerAll = kt.RandomSearch(build_model,max_trials = 5, objective="val_accuracy",directory = "AllInOne",project_name = "AllHyperParamDropout" )

In [52]:
tunerAll.search(X_train, Y_train, epochs = 5, validation_data = (X_test, Y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.7077922224998474

Best val_accuracy So Far: 0.7077922224998474
Total elapsed time: 00h 00m 15s


In [53]:
tunerAll.get_best_hyperparameters()[0].values

{'min_layers': 8,
 'units0': 56,
 'activation0': 'sigmoid',
 'Dropout0': 0.8,
 'otimizer': 'sgd',
 'units1': 8,
 'activation1': 'relu',
 'Dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'Dropout2': 0.1,
 'units3': 8,
 'activation3': 'relu',
 'Dropout3': 0.1,
 'units4': 8,
 'activation4': 'relu',
 'Dropout4': 0.1,
 'units5': 8,
 'activation5': 'relu',
 'Dropout5': 0.1,
 'units6': 8,
 'activation6': 'relu',
 'Dropout6': 0.1,
 'units7': 8,
 'activation7': 'relu',
 'Dropout7': 0.1}

In [54]:
model = tunerAll.get_best_models(num_models = 1)[0]

In [55]:
model.fit(X_train, Y_train, epochs = 200,initial_epoch = 5, validation_data = (X_test, Y_test))

Epoch 6/200
20/20 [==============================] - 2s 15ms/step - loss: 0.6961 - accuracy: 0.5326 - val_loss: 0.6789 - val_accuracy: 0.7078
Epoch 7/200
20/20 [==============================] - 0s 4ms/step - loss: 0.6895 - accuracy: 0.5684 - val_loss: 0.6732 - val_accuracy: 0.7078
Epoch 8/200
20/20 [==============================] - 0s 4ms/step - loss: 0.6838 - accuracy: 0.6107 - val_loss: 0.6679 - val_accuracy: 0.7078
Epoch 9/200
20/20 [==============================] - 0s 5ms/step - loss: 0.6795 - accuracy: 0.6156 - val_loss: 0.6633 - val_accuracy: 0.7078
Epoch 10/200
20/20 [==============================] - 0s 4ms/step - loss: 0.6741 - accuracy: 0.6384 - val_loss: 0.6584 - val_accuracy: 0.7078
Epoch 11/200
20/20 [==============================] - 0s 5ms/step - loss: 0.6741 - accuracy: 0.6221 - val_loss: 0.6537 - val_accuracy: 0.7078
Epoch 12/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6715 - accuracy: 0.6254 - val_loss: 0.6507 - val_accuracy: 0.7078
Epoch 13/